<!-- @format -->

# LLM エージェントによる Graph RAG

- LLM を使って DB クエリ（Cypher 文）を生成する方法があるが、生成される Cypher 文が一貫して正確でない可能性あり
- 代わりに、Cypher のテンプレートをセマンティックレイヤー内のツールとして実装することが提案
- LLM エージェントがそのテンプレートとやり取りできるようにする。

![alt text](https://python.langchain.com/v0.1/assets/images/graph_semantic-365248d76b7862193c33f44eaa6ecaeb.png)


In [ ]:
import os
from typing import List, Optional, Tuple, Type

from dotenv import load_dotenv
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import (
    format_to_openai_function_messages,
)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from langchain_community.graphs import Neo4jGraph
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import AzureChatOpenAI
from phoenix.trace.langchain import LangChainInstrumentor

In [ ]:
load_dotenv("../.env")

In [ ]:
os.environ["PHOENIX_PROJECT_NAME"] = "1.1 Semantic layer over graph database"
LangChainInstrumentor().instrument()

In [ ]:
graph = Neo4jGraph()

In [ ]:
description_query = """
MATCH (m:Movie|Person)
WHERE m.title CONTAINS $candidate OR m.name CONTAINS $candidate
MATCH (m)-[r:ACTED_IN|HAS_GENRE]-(t)
WITH m, type(r) as type, collect(coalesce(t.name, t.title)) as names
WITH m, type+": "+reduce(s="", n IN names | s + n + ", ") as types
WITH m, collect(types) as contexts
WITH m, "type:" + labels(m)[0] + "\ntitle: "+ coalesce(m.title, m.name)
       + "\nyear: "+coalesce(m.released,"") +"\n" +
       reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") as context
RETURN context LIMIT 1
"""


def get_information(entity: str) -> str:
    try:
        data = graph.query(description_query, params={"candidate": entity})
        return data[0]["context"]
    except IndexError:
        return "No information was found"

In [ ]:
class InformationInput(BaseModel):
    entity: str = Field(
        description="movie or a person mentioned in the question"
    )


class InformationTool(BaseTool):
    name = "Information"
    description = "useful for when you need to answer questions about various actors or movies"
    args_schema: Type[BaseModel] = InformationInput

    def _run(
        self,
        entity: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return get_information(entity)

    async def _arun(
        self,
        entity: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return get_information(entity)

<!-- @format -->

## OpenAI Agent による RAG


In [ ]:
model = "gpt-35-turbo"

llm = AzureChatOpenAI(
    azure_deployment=model,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
    temperature=0.0,
)
tools = [InformationTool()]

llm_with_tools = llm.bind(
    functions=[convert_to_openai_function(t) for t in tools]
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that finds information about movies "
            " and recommends them. If tools require follow up questions, "
            "make sure to ask the user for clarification. Make sure to include any "
            "available options that need to be clarified in the follow up questions "
            "Do only the things the user specifically requested. ",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: (
            _format_chat_history(x["chat_history"])
            if x.get("chat_history")
            else []
        ),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Who played in Casino?"})

In [ ]:
agent_executor.invoke({"input": "トム・ハンクスの出演する映画は？"})

In [ ]:
agent_executor.invoke({"input": "Tom Hanksの出演する映画は？"})

In [ ]:
agent_executor.invoke(
    {"input": "1996年に公開された映画で評価の高いものを5つ教えて"}
)